In [ ]:
# get page ranges

#  round datetime to day, get ± 7 days inclusive

#  merge any date ranges that overlaps

#  reorganise data into dict where key is date range and value is a list of pages

In [6]:
import pandas as pd
from urllib.parse import unquote
import wikitoolkit as wt
import pickle
import sqlite3
import time

In [2]:
my_agent = 'wikireddit <p.gildersleve@exeter.ac.uk>'
conn = sqlite3.connect('data/wikireddit.db')

ranges_df = pd.read_hdf('data/ranges_df.h5')

pagemapsdict = {}

In [3]:
ranges_df_consol = ranges_df.groupby(['lang', 'start_date', 'end_date'])['title'].apply(list)
starts_df_consol = ranges_df.groupby(['lang', 'start_date'])['title'].apply(list)

In [4]:
date_article_dict = {}
# convert ranges_df_consol to a dict of dicts
for lang, start_date, end_date in ranges_df_consol.index:
    if lang not in date_article_dict:
        date_article_dict[lang] = {}
    date_article_dict[lang][(start_date, end_date)] = ranges_df_consol[(lang, start_date, end_date)]

startdate_article_dict = {}
# convert starts_df_consol to a dict of dicts
for lang, start_date in starts_df_consol.index:
    if lang not in startdate_article_dict:
        startdate_article_dict[lang] = {}
    startdate_article_dict[lang][start_date] = starts_df_consol[(lang, start_date)]

Get revisions in range [from, to] (inclusive), as well as revisions at start of range

In [23]:
for lang, da_dict in date_article_dict.items():
    print(lang)
    l_revisions_df = []
    if lang not in pagemapsdict:
        pagemapsdict[lang] = wt.PageMaps()
    wtsession = wt.WTSession(f'{lang}.wikipedia', user_agent=my_agent)
    maxgroupsize = 1000
    for n, (daterange, articles) in enumerate(da_dict.items()):
        if n % 100 == 0:
            print(n/len(da_dict), end='\r')

        grouprevs = {}
        groupsize = maxgroupsize
        while len(articles) > 0:
            try:
                revisions = await wt.get_revisions(wtsession, articles[:groupsize], pagemaps=pagemapsdict[lang],
                                                start=daterange[0].isoformat(), stop=daterange[1].isoformat())
                grouprevs.update(revisions)
                articles = articles[groupsize:]
                groupsize = min(int(groupsize * (2**0.5)), maxgroupsize)
            except ValueError as e:
                # print(e, 'Reducing group size to', groupsize // 2)
                time.sleep(0.1)
                groupsize = groupsize // 2

        revisions = pd.concat({k: pd.DataFrame(v) for k, v in grouprevs.items()}).reset_index(
            level=1, drop=True).reset_index().rename(columns={'index': 'title'})
        if len(revisions) == 0:
            continue
        revisions['lang'] = lang
        revisions = revisions[['lang', 'title', 'revid', 'parentid', 'timestamp']]
        l_revisions_df.append(revisions)
        
    if len(l_revisions_df) > 0:
        l_revisions_df = pd.concat(l_revisions_df, ignore_index=True)
        l_revisions_df.to_hdf('data/revisions.h5', key=f'/{lang}', mode='a')
    await wtsession.close()

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x71b1e1bc34a0>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x71b1dc66d2b0>, 599831.462316142)])']
connector: <aiohttp.connector.TCPConnector object at 0x71b1dbbaebd0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x71b1dbbaf170>


aa
ab0
ace
ady
af0
als
am0
an0
ang
ar0
arc924170616113744
ary
arz
as0
ast
atj
avk
ay0
az0
azb
ba0
ban
bar
bat-smg


/home/patrick/Documents/R4R/.venv/lib/python3.12/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'bat-smg'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


bcl
be0
be-tarask


CancelledError: 

In [15]:
maxgroupsize = 1000
for lang, da_dict in startdate_article_dict.items():
    print(lang)
    l_revisions_df = []
    if lang not in pagemapsdict:
        pagemapsdict[lang] = wt.PageMaps()
    wtsession = wt.WTSession(f'{lang}.wikipedia', user_agent=my_agent)
    for n, (date, articles) in enumerate(da_dict.items()):
        if n % 1000 == 0:
            print(n/len(da_dict), end='\r')
        grouprevs = {}
        groupsize = maxgroupsize
        while len(articles) > 0:
            try:
                revisions = await wt.get_revision(wtsession, titles=articles[:groupsize], pagemaps=pagemapsdict[lang],
                                                 date=date.isoformat())
                grouprevs.update(revisions)
                articles = articles[groupsize:]
                groupsize = min(int(groupsize * (2**0.5)), maxgroupsize)
            except ValueError as e:
                # print(e, 'Reducing group size to', groupsize // 2)
                time.sleep(0.1)
                groupsize = groupsize // 2

        date_revisions = pd.DataFrame({k: v for k, v in grouprevs.items() if v}).T.reset_index().rename(columns={'index': 'title'})
        if len(date_revisions) == 0:
            continue
        date_revisions['lang'] = lang
        date_revisions = date_revisions[['lang', 'title', 'revid', 'parentid', 'timestamp']]
        l_revisions_df.append(date_revisions)

    if len(l_revisions_df) > 0:
        l_revisions_df = pd.concat(l_revisions_df, ignore_index=True)
        l_revisions_df.to_hdf('data/date_revisions.h5', key=f'/{lang}', mode='a')
    await wtsession.close()

aa


/tmp/ipykernel_5084/2352896156.py:34: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->Index(['lang', 'title', 'revid', 'parentid', 'timestamp'], dtype='object')]

  l_revisions_df.to_hdf('data/date_revisions.h5', key=f'/{lang}', mode='a')


ab


/tmp/ipykernel_5084/2352896156.py:34: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->Index(['lang', 'title', 'revid', 'parentid', 'timestamp'], dtype='object')]

  l_revisions_df.to_hdf('data/date_revisions.h5', key=f'/{lang}', mode='a')


ace


...

/tmp/ipykernel_5084/2352896156.py:34: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->Index(['lang', 'title', 'revid', 'parentid', 'timestamp'], dtype='object')]

  l_revisions_df.to_hdf('data/date_revisions.h5', key=f'/{lang}', mode='a')


be-tarask


/home/patrick/Documents/R4R/.venv/lib/python3.12/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'be-tarask'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/tmp/ipykernel_5084/2352896156.py:34: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->Index(['lang', 'title', 'revid', 'parentid', 'timestamp'], dtype='object')]

  l_revisions_df.to_hdf('data/date_revisions.h5', key=f'/{lang}', mode='a')


bg


CancelledError: 

In [18]:
lang = 'en'
da_dict = startdate_article_dict[lang]

print(lang)
l_revisions_df = []
if lang not in pagemapsdict:
    pagemapsdict[lang] = wt.PageMaps()
wtsession = wt.WTSession(f'{lang}.wikipedia', user_agent=my_agent)
for n, (date, articles) in enumerate(da_dict.items()):
    if n % 1000 == 0:
        print(n/len(da_dict), end='\r')
    grouprevs = {}
    groupsize = maxgroupsize
    while len(articles) > 0:
        try:
            revisions = await wt.get_revision(wtsession, titles=articles[:groupsize], pagemaps=pagemapsdict[lang],
                                                date=date.isoformat())
            grouprevs.update(revisions)
            articles = articles[groupsize:]
            groupsize = min(int(groupsize * (2**0.5)), maxgroupsize)
        except ValueError as e:
            # print(e, 'Reducing group size to', groupsize // 2)
            time.sleep(0.1)
            groupsize = groupsize // 2

    date_revisions = pd.DataFrame({k: v for k, v in grouprevs.items() if v}).T.reset_index().rename(columns={'index': 'title'})
    if len(date_revisions) == 0:
        continue
    date_revisions['lang'] = lang
    date_revisions = date_revisions[['lang', 'title', 'revid', 'parentid', 'timestamp']]
    l_revisions_df.append(date_revisions)

if len(l_revisions_df) > 0:
    l_revisions_df = pd.concat(l_revisions_df, ignore_index=True)

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7468a13a8fe0>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7468880c6d50>


en


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x74687019a990>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7468207dae70>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7468207db410>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x74681f313440>


CancelledError: 

quality

In [22]:
r_quality = await wt.get_revisions_quality(wtsession, revision_ids, 'en')
r_quality

{1236446883: {},
 1236447060: {},
 1236594994: {},
 1236598939: {},
 1236788119: {},
 1236796537: {},
 1236825393: {},
 1236947971: {},
 1236985592: {},
 1236988312: {'articlequality': 0.9955741854274109},
 1237036041: {},
 1237036430: {},
 1237103956: {},
 1237187778: {},
 1236327166: {},
 1236339025: {},
 1236340707: {},
 1236381947: {},
 1236383843: {},
 1236418369: {},
 1236418653: {},
 1236418852: {},
 1236421967: {},
 1236440853: {},
 1236457235: {},
 1236458795: {},
 1236487926: {},
 1236488507: {},
 1236488568: {},
 1236515418: {},
 1236537112: {},
 1236585466: {},
 1236587924: {},
 1236588591: {},
 1236588814: {},
 1236596064: {},
 1236599399: {},
 1236603845: {},
 1236619389: {},
 1236620809: {},
 1236629366: {},
 1236632371: {},
 1236670767: {},
 1236709239: {},
 1236711950: {},
 1236794890: {},
 1236860243: {},
 1236860407: {},
 1236861070: {},
 1236861338: {},
 1236861759: {},
 1236861845: {},
 1236870738: {},
 1236871607: {},
 1236871701: {},
 1236943423: {},
 1236943717:

topics

In [14]:
len(da_dict)

276

In [20]:
wt.api_article_views(wtsession, f'{lang}.wikipedia', ['London'], pagemaps=pagemapsdict[lang],
                                         start=daterange[0].strftime('%Y%m%d'), end=daterange[1].strftime('%Y%m%d'))

{datetime.datetime(2021, 2, 5, 0, 0): {'London': 6},
 datetime.datetime(2021, 2, 6, 0, 0): {'London': 3},
 datetime.datetime(2021, 2, 7, 0, 0): {'London': 1},
 datetime.datetime(2021, 2, 8, 0, 0): {'London': 2},
 datetime.datetime(2021, 2, 9, 0, 0): {'London': 3},
 datetime.datetime(2021, 2, 10, 0, 0): {'London': 0},
 datetime.datetime(2021, 2, 11, 0, 0): {'London': 3},
 datetime.datetime(2021, 2, 12, 0, 0): {'London': 1},
 datetime.datetime(2021, 2, 13, 0, 0): {'London': 0},
 datetime.datetime(2021, 2, 14, 0, 0): {'London': 39},
 datetime.datetime(2021, 2, 15, 0, 0): {'London': 84},
 datetime.datetime(2021, 2, 16, 0, 0): {'London': 62},
 datetime.datetime(2021, 2, 17, 0, 0): {'London': 43},
 datetime.datetime(2021, 2, 18, 0, 0): {'London': 33},
 datetime.datetime(2021, 2, 19, 0, 0): {'London': 23},
 datetime.datetime(2021, 2, 20, 0, 0): {'London': 14},
 datetime.datetime(2021, 2, 21, 0, 0): {'London': 8},
 datetime.datetime(2021, 2, 22, 0, 0): {'London': 14},
 datetime.datetime(2021, 

pageviews

In [16]:
final_pvdf = pd.DataFrame()
lc = 0
for lang, da_dict in date_article_dict.items():
    if lang not in pagemapsdict:
        pagemapsdict[lang] = wt.PageMaps()
    wtsession = wt.WTSession(f'{lang}.wikipedia', user_agent=my_agent)
    for daterange, articles in da_dict.items():
        # check for end of day!!!!
        pageviews = wt.api_article_views(wtsession, f'{lang}.wikipedia', articles, pagemaps=pagemapsdict[lang],
                                         start=daterange[0].strftime('%Y%m%d'), end=daterange[1].strftime('%Y%m%d'))
        pvdf = pd.DataFrame(pageviews).T.reset_index().rename(columns={'index': 'date'})
        pvdf = pvdf.melt(id_vars='date', var_name='title', value_name='pageviews')
        pvdf['lang'] = lang
        pvdf = pvdf[['lang', 'title', 'date', 'pageviews']]
        final_pvdf = pd.concat([final_pvdf, pvdf])
    await wtsession.close()
    
    lc+=1
    if lc > 5:
        break

final_pvdf

ERROR while fetching and parsing ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/ar.wikipedia/all-access/all-agents/%D9%85%D9%82%D9%88/daily/2021020500/2021022500']


Traceback (most recent call last):
  File "/home/patrick/Documents/R4R/.venv/lib/python3.12/site-packages/mwviews/api/pageviews.py", line 146, in article_views
    raise Exception(
Exception: The pageview API returned nothing useful at: ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/ar.wikipedia/all-access/all-agents/%D9%85%D9%82%D9%88/daily/2021020500/2021022500']


Exception: The pageview API returned nothing useful at: ['https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/ar.wikipedia/all-access/all-agents/%D9%85%D9%82%D9%88/daily/2021020500/2021022500']